<a href="https://colab.research.google.com/github/alexandrekj/projeto_EMB/blob/main/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# building gdrive and setting path 

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/TCC/mobilefacenet-zhijiesun
%ls

Mounted at /content/drive
/content/drive/MyDrive/TCC/mobilefacenet-zhijiesun
complete_model/  mobilefacenet.h5      model.tf/       run.ipynb  valid/
dataset/         model_arcface_new.h5  my_h5_model.h5  test/      weights.h5


In [34]:
!pip install keras==2.12.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.11.0
    Uninstalling keras-2.11.0:
      Successfully uninstalled keras-2.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.3 which is incompatible.


In [2]:
import keras 
print(keras.__version__)

2.12.0


In [16]:
# 2nd arcface loss implementation
# arcFace Loss by  https://github.com/godofpdog/MobileFaceNet_keras/blob/master/src/build_model.py

import math

class ArcFaceLossLayer(Layer):
    def __init__(self, s=64, m=0.5, num_classes=24, is_use_bais=False, **kwargs):
        self.s = float(s)
        self.m = float(m)
        self.num_classes = num_classes
        self.is_use_bais = is_use_bais
        super(ArcFaceLossLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self._weights = self.add_weight(name='Weights',
                                        shape=(input_shape[0][1], self.num_classes), 
                                        initializer='he_normal',
                                        regularizer=None,
                                        trainable=True,
                                        constraint=None
                                       )
        # input_shape[0][1]
        # shape=(input_shape[0][1], self.num_classes
        super(ArcFaceLossLayer, self).build(input_shape)
         
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.num_classes)

    def call(self, _inputs):
        # ** labels is one-hot format
        embedding_input, labels = _inputs
        s = self.s 
        m = self.m 
        num_classes = self.num_classes 
        cos_m = math.cos(m)
        sin_m = math.sin(m)
        thresh = math.cos(math.pi - m)

        # ** l2 normalization 
        embedding_norm = tf.norm(embedding_input, axis=1, keepdims=True)
        embedding = tf.divide(embedding_input, embedding_norm, name='norm_embedding')
        weights_norm = tf.norm(self._weights, axis=0, keepdims=True)
        weights = tf.divide(self._weights, weights_norm, name='norm_weights')

        # ** cos(theta + m)
        cos_theta = tf.matmul(embedding, weights, name='cos_theta')
        cos_theta_square = tf.square(cos_theta, name='cos_theta_square')
        sin_theta_square = tf.subtract(1.0, cos_theta_square, name='sin_theta_square')
        sin_theta = tf.sqrt(sin_theta_square, name='sin_theta')
        cos_theta_m = s * tf.subtract(tf.multiply(cos_theta, cos_m), tf.multiply(sin_theta, sin_m), name='cos_theta_m')

        # ** this condition controls the theta+m should in range [0, pi]
        # **  0 <= theta + m <= pi
        # ** -m <= theta + 0 <= pi - m
        cond_value = cos_theta - thresh 
        cond_index = tf.cast(tf.nn.relu(cond_value, name='if_else'), dtype=tf.bool)
        keep_value = s * (cos_theta - sin_m * m)
        cos_theta_m_adj = tf.where(cond_index, cos_theta_m, keep_value)
        s_cos_theta = tf.multiply(s, cos_theta, name='scalar_cos_theta')

        # **
        mask = labels
        inv_mask = tf.subtract(1., mask, name='inverse_mask')

        # ** logit
        logit = tf.add(tf.multiply(s_cos_theta, inv_mask), tf.multiply(cos_theta_m_adj, mask), name='arcface_loss_output')
        
        # ** loss
        self.inference_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logit, labels=labels))

        return self.inference_loss



In [7]:
# Inspired by implementation on https://github.com/zhijiesun/Mobilefacenet/tree/master

### Arcface code ####

from keras import backend as K
from keras.layers import Layer
from keras import regularizers

import tensorflow as tf

class ArcFace(Layer):
  def __init__(self, n_classes=10, s=30.0, m=0.50, regularizer=None, **kwargs):
    super(ArcFace, self).__init__(**kwargs)
    self.n_classes = n_classes
    self.s = s
    self.m = m
    self.regularizer = regularizers.get(regularizer)

  def build(self, input_shape):
    super(ArcFace, self).build(input_shape[0])
    self.W = self.add_weight(name='W',
                             shape=(input_shape[0][-1], self.n_classes),
                             initializer='glorot_uniform',
                             trainable=True,
                             regularizer=self.regularizer)
 
  def call(self, inputs):
    x, y = inputs
    c = K.shape(x)[-1]
    # normalize feature and weights
    x = tf.nn.l2_normalize(x,dim=1)
    W = tf.nn.l2_normalize(self.W,dim=0)
    #dot product
    logits = x @ W
    # add margin
    # clip logits to prevent zero division when backward
    theta = tf.acos(K.clip(logits, -1.0 + K.epsilon(), 1.0 - K.epsilon()))
    target_logits = tf.cos(theta + self.m)
    # sin = tf.sqrt(1 - logits**2)
    # cos_m = tf.cos(logits)
    # sin_m = tf.sin(logits)
    # target_logits = logits * cos_m - sin * sin_m
    #
    logits = logits * (1 - y) + target_logits * y
    # feature re-scale
    logits *= self.s
    out = tf.nn.softmax(logits)

    return out 

  def compute_output_shape(self, input_shape):
        return (None, self.n_classes)

  def get_config(self):
        config = super(ArcFace, self).get_config()
        config.update({
            "n_classes": self.n_classes,
            "s": self.s,
            "m": self.m,
            "regularizer": self.regularizer,
        })
        return config



Mobilefacenet architecture definition

In [3]:
# Mobilefacenet architecture

from keras.regularizers import l2
from keras.models import Model
from keras.layers import *
import tensorflow as tf


weight_decay=0.0005
def InvertedResidualBlock(x, expand, out_channels, repeats, stride, weight_decay, block_id):

    '''
    This function defines a sequence of 1 or more identical layers, referring to Table 2 in the original paper.
    :param x: Input Keras tensor in (B, H, W, C_in)
    :param expand: expansion factor in bottlenect residual block
    :param out_channels: number of channels in the output tensor
    :param repeats: number of times to repeat the inverted residual blocks including the one that changes the dimensions.
    :param stride: stride for the 1x1 convolution
    :param weight_decay: hyperparameter for the l2 penalty
    :param block_id: as its name tells
    :return: Output tensor (B, H_new, W_new, out_channels)
    '''

    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
    in_channels = K.int_shape(x)[channel_axis]
    x = Conv2D(expand * in_channels, 1, padding='same', strides=stride, use_bias=False, kernel_regularizer=l2(weight_decay), name='conv_%d_0' % block_id)(x)
    x = BatchNormalization()(x)
    x = PReLU(shared_axes=[1,2])(x)
    x = DepthwiseConv2D((3, 3), padding='same',depth_multiplier=1,strides=1,use_bias=False,kernel_regularizer=l2(weight_decay),name='conv_dw_%d_0' % block_id )(x)
    x = BatchNormalization()(x)
    x = PReLU(shared_axes=[1,2])(x)
    x = Conv2D(out_channels, 1, padding='same', strides=1, use_bias=False, kernel_regularizer=l2(weight_decay), name='conv_bottleneck_%d_0' % block_id)(x)
    x = BatchNormalization()(x)

    for i in range(1, repeats):
        x1 = Conv2D(expand*out_channels, 1, padding='same', strides=1, use_bias=False, kernel_regularizer=l2(weight_decay), name='conv_%d_%d' % (block_id, i))(x)
        x1 = BatchNormalization()(x1)
        x1 = PReLU(shared_axes=[1,2])(x1)
        x1 = DepthwiseConv2D((3, 3), padding='same', depth_multiplier=1,strides=1,use_bias=False,kernel_regularizer=l2(weight_decay), name='conv_dw_%d_%d' % (block_id, i))(x1)
        x1 = BatchNormalization()(x1)
        x1 = PReLU(shared_axes=[1,2])(x1)
        x1 = Conv2D(out_channels, 1, padding='same', strides=1, use_bias=False, kernel_regularizer=l2(weight_decay),name='conv_bottleneck_%d_%d' % (block_id, i))(x1)
        x1 = BatchNormalization(axis=channel_axis, epsilon=1e-5, momentum=0.9, name='conv_bottlenet_%d_%d_bn' % (block_id, i))(x1)
        x = Add()([x, x1])

    return x

def regular_conv(x, filters, ks, p, s, k_init, use_b=False):
  x = Conv2D(filters, ks, padding=p, strides=s, kernel_initializer=k_init, kernel_regularizer=regularizers.l2(weight_decay), use_bias=use_b)(x)
  x = BatchNormalization()(x)
  x = PReLU(shared_axes=[1,2])(x)
  return x

def depth_conv(x, filters, ks, p, s, k_init, use_b=False):
  x = DepthwiseConv2D(ks, padding=p, strides=s, kernel_initializer=k_init, kernel_regularizer=regularizers.l2(weight_decay), use_bias=use_b)(x)
  x = BatchNormalization()(x)
  x = PReLU(shared_axes=[1,2])(x)
  return x


def valid_conv(x, filters, ks, p, s, k_init, use_b=False):
  x = Conv2D(filters, ks, padding=p, strides=s,kernel_initializer=k_init, kernel_regularizer=regularizers.l2(weight_decay), use_bias=use_b)(x)
  x = BatchNormalization()(x)
  x = PReLU(shared_axes=[1,2])(x)
  return x


def mobilefacenet():
    input = Input(shape=(112, 112, 3))
    # parameter order: filters_num, kernel_size, padding, stride, kernel_initializer, bias_term
    x = regular_conv(input, 64, (3, 3), 'same', (2, 2), 'glorot_uniform', False)
    x = depth_conv(x, 64, (3, 3), 'valid', (1, 1), 'glorot_uniform', False)
    x = InvertedResidualBlock(x, 2,  64, 5, (2,2),  weight_decay, 1)
    x = InvertedResidualBlock(x, 4, 128, 1, (2, 2), weight_decay, 2)
    x = InvertedResidualBlock(x, 2, 128, 6, (1, 1), weight_decay, 3)
    x = InvertedResidualBlock(x, 4, 128, 1, (2, 2), weight_decay, 4)
    x = InvertedResidualBlock(x, 2, 128, 2, (1, 1), weight_decay, 5)
    x = valid_conv(x, 512, (1, 1), 'valid', (1, 1), 'glorot_uniform', False)
    x = depth_conv(x, 512, (7, 6), 'valid', (1, 1), 'glorot_uniform', False)
    x = valid_conv(x, 128, (1, 1), 'valid', (1, 1), 'glorot_uniform', False)

    print(x.shape)

    x = Flatten()(x)
    output = Dense(128, use_bias=False, kernel_initializer='glorot_uniform')(x)

    return Model(input, output)

def mobilefacenet_arcface():
    input = Input(shape=(112, 112, 3))
    y = Input(shape=(51,))
    # parameter order: filters_num, kernel_size, padding, stride, kernel_initializer, bias_term
    x = regular_conv(input, 64, (3, 3), 'same', (2, 2), 'glorot_uniform', False)
    x = depth_conv(x, 64, (3, 3), 'valid', (1, 1), 'glorot_uniform', False)
    x = InvertedResidualBlock(x, 2, 64, 5, (2, 2), weight_decay, 1)
    x = InvertedResidualBlock(x, 4, 128, 1, (2, 2), weight_decay, 2)
    x = InvertedResidualBlock(x, 2, 128, 6, (1, 1), weight_decay, 3)
    x = InvertedResidualBlock(x, 4, 128, 1, (2, 2), weight_decay, 4)
    x = InvertedResidualBlock(x, 2, 128, 2, (1, 1), weight_decay, 5)
    x = valid_conv(x, 512, (1, 1), 'valid', (1, 1), 'glorot_uniform', False)
    x = depth_conv(x, 512, (7, 6), 'valid', (1, 1), 'glorot_uniform', False)
    x = valid_conv(x, 128, (1, 1), 'valid', (1, 1), 'glorot_uniform', False)
    x = Flatten()(x)
    x = Dense(128, use_bias=False, kernel_initializer='glorot_uniform')(x)
    output = ArcFace(51, regularizer=regularizers.l2(weight_decay))([x, y])
    #output = ArcFaceLossLayer(s=64, m=0.5, num_classes=51, is_use_bais=False)([x, y])


    return Model([input, y], output)



Trying to implement the quantization aware training


In [4]:
!pip install -q tensorflow-model-optimization

Post training quantization

In [15]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

In [18]:
# Saving corresponding quantized model
with open('quantized_model.tflite', 'wb') as f:
  f.write(tflite_quant_model)

In [8]:
## Train code

import keras
import os, shutil
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

train_dir = './dataset'
#valid_dir = './test'
#test_dir = './valid'

#model=mobilefacenet_arcface()
model = mobilefacenet()
model.summary()



''' 
DONT for now

train_datagen = ImageDataGenerator(rescale=1./255) #rescale the tensor values to [0,1]
train_image = train_datagen.flow_from_directory(
         train_dir,
         target_size=(112, 112),
         color_mode="rgb",
         batch_size=32,
         class_mode='categorical')

valid_generator = valid_datagen.flow_from_directory(
    directory= valid_dir,
    target_size=(112, 112),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical")

test_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(112, 112),
    color_mode="rgb",
    batch_size=1,
    class_mode=None
)
'''


(None, 1, 2, 128)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 112, 112, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 56, 56, 64)   1728        ['input_2[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 56, 56, 64)  256         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                            

' \nDONT for now\n\ntrain_datagen = ImageDataGenerator(rescale=1./255) #rescale the tensor values to [0,1]\ntrain_image = train_datagen.flow_from_directory(\n         train_dir,\n         target_size=(112, 112),\n         color_mode="rgb",\n         batch_size=32,\n         class_mode=\'categorical\')\n\nvalid_generator = valid_datagen.flow_from_directory(\n    directory= valid_dir,\n    target_size=(112, 112),\n    color_mode="rgb",\n    batch_size=32,\n    class_mode="categorical")\n\ntest_generator = test_datagen.flow_from_directory(\n    directory=test_dir,\n    target_size=(112, 112),\n    color_mode="rgb",\n    batch_size=1,\n    class_mode=None\n)\n'

In [ ]:
# DONT!
# New training (based on https://vijayabhaskar96.medium.com/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_VALID)

# Predict the output

STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

predicted_class_indices=np.argmax(pred,axis=1)

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)




In [30]:
# Adapted from mobilenet-FD by ST

validation_split = 0.9
batch_size = 256

nb_train_files = sum([len(files) for i, j, files in os.walk("./dataset")])
print("Total number of samples in train folder {}".format(nb_train_files))

train_datagen = ImageDataGenerator(rescale=1./255,validation_split=validation_split) #rescale the tensor values to [0,1]
train_generator = train_datagen.flow_from_directory('./dataset',
                                               target_size=(112, 112),
                                               batch_size=batch_size,
                                               class_mode='categorical',
                                               subset='training')

validation_generator = train_datagen.flow_from_directory('./dataset',
                                                    target_size=(112, 112),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    subset='validation')


checkpointer = ModelCheckpoint('mobilefacenet.h5',
                               monitor='val_acc',
                               save_best_only=True,
                               verbose=1)

# Compile the model
# previously : model.compile(loss='categorical_crossentropy',
model.compile(loss=dummy_loss,
              optimizer=optimizers.SGD(lr=0.01),
              metrics=['acc'])




epochs=2
for e in range(epochs):
    print('Epoch', e)
    for i in range(len(train_generator)):
        X_batch, label_batch = train_generator[i]
        print(len(X_batch))
        #model.train_on_batch([X_batch, label_batch],label_batch) with mobilefacenet_arcface
        model.train_on_batch(X_batch, label_batch)



model.save("model_arcface_new.h5")

'''

history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              steps_per_epoch=int( (nb_train_files*(1-validation_split)) / batch_size),
                              validation_steps=int( (nb_train_files*(validation_split)) / batch_size),
                              callbacks=[checkpointer])
'''

Total number of samples in train folder 9437
Found 966 images belonging to 51 classes.
Found 8471 images belonging to 51 classes.
Epoch 0


/usr/local/lib/python3.10/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


256
256
256
198
Epoch 1
256
256
256
198


'\n\nhistory = model.fit_generator(train_generator,\n                              epochs=10,\n                              validation_data=validation_generator,\n                              steps_per_epoch=int( (nb_train_files*(1-validation_split)) / batch_size),\n                              validation_steps=int( (nb_train_files*(validation_split)) / batch_size),\n                              callbacks=[checkpointer])\n'

In [31]:
new_model = tf.keras.models.load_model('model_arcface_new.h5', custom_objects={'ArcFace': ArcFace})
print(new_model.summary())
new_model.save("my_h5_model.h5")

### Search on L2 custom object

ValueError: ignored

In [8]:
model.save("my_h5_model.h5")

In [ ]:
# DONT 
# Old training scirpt

model.compile(loss='categorical_crossentropy',
            optimizer=optimizers.SGD(lr=0.01),
            metrics=['acc'])


epochs=10
for e in range(epochs):
    print('Epoch', e)
    for i in range(len(train_image)):
        X_batch, label_batch = train_image[i]
        print(len(X_batch))
        model.train_on_batch([X_batch, label_batch],label_batch)


check_pointer = ModelCheckpoint(filepath = 'model.h5', verbose = 1)


  
